<h1 align="center" style="color:#96d9f0;">Computational Intelligence for
Optimization - Project</h1>
<h3 align="center" style="color:#96d9f0;">Group AW - Wedding Seating Optimization</h3>

---

### <span style="color:#96d9f0;">Group Members</span>

<table>
  <thead style="color:#96d9f0;">
    <tr>
      <th>Name</th>
      <th>Email</th>
      <th>Student ID</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Afonso Dias</td>
      <td>20211540@novaims.unl.pt</td>
      <td>20211540</td>
    </tr>
    <tr>
      <td>Isabel Duarte</td>
      <td>20240545@novaims.unl.pt</td>
      <td>20240545</td>
    </tr>
    <tr>
      <td>Pedro Campino</td>
      <td>20240537@novaims.unl.pt</td>
      <td>20240537</td>
    </tr>
    <tr>
      <td>Rita Matos</td>
      <td>20211642@novaims.unl.pt</td>
      <td>20211642</td>
    </tr>
  </tbody>
</table>

In [1]:
#future improvements: test random fitnesses to set a baseline average to compare our results

In [2]:
#No swap mutation within the same table

---

### <span style="color:#96d9f0;">1. Imports</span>

This section includes all the necessary library imports required for the development of the project. We also use the library provided during the practical classes, as recommended, to support the implementation. Additionally, the relationship matrix used for evaluating guest compatibility is imported here.

In [1]:
import sys
sys.path.append('..')

In [13]:
import random
from copy import deepcopy
from library.solution import Solution
import csv
import os
import pandas as pd
import numpy as np
from itertools import combinations

In [14]:
# Import the relationship matrix as a pandas dataframe for visualization

relationship_matrix_df = pd.read_csv('data/seating_data.csv')
relationship_matrix_df.drop(columns='idx', inplace=True)
relationship_matrix_df

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,0,5000,0,0,700,700,0,0,0,0,...,100,100,0,0,100,100,100,0,0,0
1,5000,0,700,700,0,0,300,300,500,500,...,100,100,0,100,0,0,0,0,0,0
2,0,700,0,2000,0,0,0,0,300,300,...,0,0,0,0,0,0,0,0,0,0
3,0,700,2000,0,0,0,900,400,300,300,...,0,0,0,0,0,0,0,0,0,0
4,700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,100,0,0,2000,700,700
61,0,0,0,0,0,0,0,0,0,0,...,0,0,-1000,0,100,0,2000,0,700,700
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,700,700,0,900


In [15]:
# Import the relationship matrix as a numpy array

relationship_matrix = np.loadtxt('data/seating_data.csv', delimiter=',', skiprows=1)
relationship_matrix = relationship_matrix[ : , 1:]
relationship_matrix 

array([[   0., 5000.,    0., ...,    0.,    0.,    0.],
       [5000.,    0.,  700., ...,    0.,    0.,    0.],
       [   0.,  700.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,  700.,  700.],
       [   0.,    0.,    0., ...,  700.,    0.,  900.],
       [   0.,    0.,    0., ...,  700.,  900.,    0.]], shape=(64, 64))

---

### <span style="color:#96d9f0;">2. Solution Representation</span>

In this section, we define the Wedding Sitting Solution class (`WSSolution`), which extends the base `Solution` class provided in the practical classes. This class represents a candidate solution for the wedding seating optimization problem.

Each solution consists of a list of tables, where each table is a list of guest indices. The class includes methods to generate a valid random initial representation and to evaluate the fitness of a solution based on the total relationship scores between guests seated at the same table.

In [18]:
class WSSolution(Solution):
    def __init__(self, repr=None, relationship_matrix=relationship_matrix, nr_of_tables=8):
        self.relationship_matrix = relationship_matrix
        self.nr_of_tables = nr_of_tables
        self.table_capacity = len(relationship_matrix) // nr_of_tables
        super().__init__(repr=repr)

    def random_initial_representation(self):
        tables = []
        left_idxs = [idx for idx in range(len(self.relationship_matrix))]
        for i in range(self.nr_of_tables):
            tables.append([])
            for j in range(self.table_capacity):
                idx = random.choice(left_idxs)
                left_idxs.remove(idx)
                # Check if idx is already in a table
                while any(idx in table for table in tables):
                    idx = random.randint(0, len(self.relationship_matrix) - 1)
                tables[i].append(idx)
        
        return tables
    
    def fitness(self):
        total_happiness = 0
        for i, row in enumerate(self.repr):
            table_happiness = 0
            combs = list(combinations(row, 2))
            pair_scores = [self.relationship_matrix[a, b] for a, b in combs]
            table_happiness = sum(s for s in pair_scores)
            total_happiness += table_happiness
        
        return total_happiness


In [19]:
solution = WSSolution()

print('Random solution:', solution)
print('Total Fitness:', solution.fitness())

Random solution: [[40, 39, 57, 9, 28, 6, 15, 19], [16, 52, 11, 44, 47, 31, 8, 48], [42, 63, 4, 45, 7, 33, 2, 13], [5, 60, 14, 56, 50, 59, 22, 58], [51, 34, 43, 36, 29, 27, 32, 25], [20, 46, 10, 23, 26, 3, 49, 55], [30, 24, 1, 18, 37, 38, 62, 17], [12, 53, 41, 54, 21, 61, 35, 0]]
Total Fitness: 5200.0


---

### <span style="color:#96d9f0;">3. Selection Algorithms</span>

#### Fitness Proportionate Selection

In [ ]:
# Code (Afonso)

#### Ranking Selection

In [ ]:
# Code (Isa)

#### Tournament Selection

In [21]:
def tournament_selection(population, k=3):
    competitors = random.sample(population, k)
    return max(competitors, key=lambda ind: ind.fitness())

---

### <span style="color:#96d9f0;">4. Crossovers</span>

#### Standard Crossover

In [ ]:
# Code (Afonso)

#### Partially Matched Crossover

In [ ]:
# Code (Isa)

#### Order Preserving Crossover

In [ ]:
# Code (Rita)

#### Group Preserving Crossover

In [ ]:
# Code (Pedro)

---

### <span style="color:#96d9f0;">5. Mutations</span>

#### Swap Mutation

In [ ]:
# Swap mutation implementation that swaps two random people between two different tables and does not allow swapping within the same table

def swap_mutation(solution, mut_prob):
    if random.random() < mut_prob:
        tables = solution.repr
        if len(tables) < 2:
            return

        # Pick two random tables
        t1, t2 = random.sample(range(len(tables)), 2)
        if not tables[t1] or not tables[t2]:
            return

        # Swap one person from each
        i1 = random.randint(0, len(tables[t1]) - 1)
        i2 = random.randint(0, len(tables[t2]) - 1)
        tables[t1][i1], tables[t2][i2] = tables[t2][i2], tables[t1][i1]

#### New Creative Mutation

In [ ]:
# Code (Afonso)